In [13]:
# implementation of a character-based RNN to generate sonnets
import numpy as np
import random
import string
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, Lambda
from keras.layers import SimpleRNN
from keras.callbacks import LambdaCallback, EarlyStopping

In [3]:
def preprocess(filename="shakespeare.txt", seq_length=40, step=5):
    '''
    returns semi-redundant sequences their outputs 
    seq_length: number of characters in each sequence
    step: gets every [step] sequence  
    '''

    # puts all data into text string  
    file = open(filename, "r")
    text = ""
    for line in file:
        line = line.lstrip(' ').rstrip(' ')
        if line != '\n' and not line[0].isdigit():
            line.translate(str.maketrans('', '', string.punctuation))
            text += line.lower()

    # make char to index and index to char dictionary 
    characters = sorted(list(set(text)))
    char_indices_dict = dict((c, i) for i, c in enumerate(characters))
    indices_char_dict = dict((i, c) for i, c in enumerate(characters))
    #print(char_indices_dict)

    # makes every [step] char sequences of length seq_length and their outputs
    sequences = []
    next_chars = [] # next char that seq in sequences generates
    #print(repr(text[len(text) - 200:]))
    for i in range(0, len(text) - seq_length, step):
        #print(i, seq, text[i : i + seq_length])
        sequences.append(text[i : i + seq_length])
        next_chars.append(text[i + seq_length])

    # put sequences and outputs into np array
    x = np.zeros((len(sequences), seq_length, len(characters)))
    y = np.zeros((len(sequences), len(characters)), dtype=np.bool)
    for i, sequence in enumerate(sequences):
        for t, char in enumerate(sequence):
            x[i, t, char_indices_dict[char]] = 1
        y[i, char_indices_dict[next_chars[i]]] = 1

    return x, y, sequences, indices_char_dict, char_indices_dict, text

In [4]:
#seq_length: number of characters in each sequence
#step: gets every [step] sequence  

seq_length=40
step=5    
filename="shakespeare.txt"
# puts all data into text string  
file = open(filename, "r")
text = ""
for line in file:
    line = line.lstrip(' ').rstrip(' ')
    if line != '\n' and not line[0].isdigit():
        line.translate(str.maketrans('', '', string.punctuation))
        text += line.lower()

In [5]:
# make char to index and index to char dictionary 
characters = sorted(list(set(text)))
char_indices_dict = dict((c, i) for i, c in enumerate(characters))
indices_char_dict = dict((i, c) for i, c in enumerate(characters))
print(char_indices_dict)

{'\n': 0, ' ': 1, '!': 2, "'": 3, '(': 4, ')': 5, ',': 6, '-': 7, '.': 8, ':': 9, ';': 10, '?': 11, 'a': 12, 'b': 13, 'c': 14, 'd': 15, 'e': 16, 'f': 17, 'g': 18, 'h': 19, 'i': 20, 'j': 21, 'k': 22, 'l': 23, 'm': 24, 'n': 25, 'o': 26, 'p': 27, 'q': 28, 'r': 29, 's': 30, 't': 31, 'u': 32, 'v': 33, 'w': 34, 'x': 35, 'y': 36, 'z': 37}


In [6]:
# makes every [step] char sequences of length seq_length and their outputs
sequences = []
next_chars = [] # next char that seq in sequences generates
print(repr(text[len(text) - 200:]))
for i in range(0, len(text) - seq_length, step):
    sequences.append(text[i : i + seq_length])
    next_chars.append(text[i + seq_length])

"fire took heat perpetual,\ngrowing a bath and healthful remedy,\nfor men discased, but i my mistress' thrall,\ncame there for cure and this by that i prove,\nlove's fire heats water, water cools not love."


In [7]:

# put sequences and outputs into np array
x = np.zeros((len(sequences), seq_length, len(characters)))
y = np.zeros((len(sequences), len(characters)), dtype=np.bool)
for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i, t, char_indices_dict[char]] = 1
    y[i, char_indices_dict[next_chars[i]]] = 1

In [19]:
# using rnn to build model
def make_model(temperature=1.0):
    model = Sequential()
    model.add(SimpleRNN(100))
    # add temperature (controls variance)
    model.add(Lambda(lambda x: x / temperature))
    model.add(Dense(len(indices_char_dict), activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    earlyStopping = EarlyStopping(monitor='loss', patience=3, verbose=1, mode='auto')
    return model

In [23]:
x, y, sequences, indices_char_dict, char_indices_dict, text = preprocess()
model=make_model(0.25)
model.build(x.shape)
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_6 (SimpleRNN)     (18727, 100)              13900     
_________________________________________________________________
lambda_6 (Lambda)            (18727, 100)              0         
_________________________________________________________________
dense_6 (Dense)              (18727, 38)               3838      
Total params: 17,738
Trainable params: 17,738
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.fit(x, y, epochs=50, verbose=1, callbacks=[earlyStopping])
model.save('rnn.h5')

Epoch 1/50
586/586 [==============================] - 19s 30ms/step - loss: 2.9217 - accuracy: 0.2102
Epoch 2/50
586/586 [==============================] - 17s 29ms/step - loss: 2.2850 - accuracy: 0.3411
Epoch 3/50
394/586 [===================>..........] - ETA: 5s - loss: 2.1864 - accuracy: 0.3552

In [ ]:
def generate_sonnet():
    x, y, sequences, indices_char_dict, char_indices_dict, text = preprocess()

    model = load_model('rnn.h5')
    sonnet = []


    seq = "shall i compare thee to a summer's day?\n"
    sonnet.append(seq)
    for _ in range(13):
        line = ""
        for i in range(40):
            x = np.zeros((1, len(seq), len(indices_char_dict)))
            for t, index in enumerate(seq):
                x[0, t, char_indices_dict[index]] = 1.

            prediction = model.predict(x, verbose=0)[0]
            index = np.argmax(prediction)
            char = indices_char_dict[index]
            line += char
            seq = seq[1:] + char

        sonnet.append(line)
    return sonnet

In [ ]:
sonnet=generate_sonnet()

for line in sonnet:
    print(line)